In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv('../input/dataset/Amazon alexa review - code/dataset/amazon_alexa.tsv',sep ='\t')
data.head()

In [3]:
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')

In [42]:
data.head()

In [5]:
data.isnull().sum()

In [6]:
data.info()

In [7]:
data['variation'].value_counts()

In [8]:
sns.countplot(data['variation'])

In [9]:
data['feedback'].value_counts()

In [10]:
sns.countplot(data['rating'],hue = data['feedback'])

In [11]:
data.groupby(['rating'])['feedback'].value_counts()

# it was obvious to have positive feedback for ratings more than 3 

In [12]:
len(data)

In [13]:
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [14]:
review = []
for i in range(0,len(data)):
    word = re.sub('[^a-zA-Z]' , ' ' ,data['verified_reviews'][i])
    #data['verified_reviews']= data['verified_reviews'][i].lower()
    word = word.lower()
    word = word.split()
    #word = nltk.word_tokenize(word)
    word = [ps.stem(x) for x in word if not x in stopwords.words('english') ]
    word = ' '.join(word)
    review.append(word)
    

In [15]:
#review

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['variation'] = le.fit_transform(data['variation'])
#data

In [17]:
data['verified_reviews'] = review

In [18]:
from keras.preprocessing.text import Tokenizer


In [19]:
data.shape

In [20]:

token = Tokenizer(3105,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
token.fit_on_texts(data['verified_reviews'])
review = token.texts_to_sequences(review)

In [21]:
from keras.preprocessing.sequence import pad_sequences
review = pad_sequences(review,maxlen =80,padding ='post')

In [22]:
x=review
y=data.feedback

In [23]:
x.shape

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2,random_state =42)


In [25]:
randomforest = RandomForestClassifier()
randomforest.fit(x_train,y_train)

In [26]:
randomforest.score(x_test,y_test)

In [27]:
predict=randomforest.predict(x_test)

In [44]:
def final_result(input):
    review = []
    for i in range(0,1):
        word = re.sub('[^a-zA-Z]' , ' ' ,input)
        word = word.lower()
        word = word.split()
    
        word = [ps.stem(x) for x in word if not x in stopwords.words('english') ]
        word = ' '.join(word)
        review.append(word)
        
    token = Tokenizer(len(input),filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    token.fit_on_texts(review)
    review = token.texts_to_sequences(review)
    
    review = pad_sequences(review,maxlen =80,padding ='post')
    
    predict=randomforest.predict(review)
    
    if predict.all() ==1:
        print('feedback is positive')
    else:
        print('feedback is negative')
        

In [47]:

#final_result(str(input("enter feedback : ")))